# AOG Speech-to-Text Example

This notebook demonstrates how to use AOG Speech-to-Text API to convert speech to text.

## Table of Contents
1. [Environment Setup](#1-Environment Setup)
2. [Helper Functions](#2-Helper Functions)
3. [Basic Speech Recognition](#3-Basic Speech Recognition)
4. [Multi-language Recognition](#4-Multi-language Recognition)
5. [Timeline Processing](#5-Timeline Processing)

## 1. Environment Setup

In [ ]:
import requests
import base64
import json
from typing import Optional, Dict, List
from pathlib import Path
from IPython.display import Audio, display

# AOG service configuration
AOG_BASE_URL = "http://localhost:16688"
AOG_API_SPEC_VERSION = "v0.2"
DEFAULT_MODEL = "NamoLi/whisper-large-v3-ov"

print("✅ Configuration complete")
print(f"📍 AOG service address: {AOG_BASE_URL}")
print(f"🤖 Default model: {DEFAULT_MODEL}")

## 2. Helper Functions

In [ ]:
def audio_to_base64(audio_path: str) -> Optional[str]:
    """
    将音频文件转换为 base64 编码
    
    Args:
        audio_path: 音频文件路径
    
    Returns:
        base64 编码字符串
    """
    try:
        with open(audio_path, "rb") as f:
            audio_data = f.read()
            return base64.b64encode(audio_data).decode()
    except Exception as e:
        print(f"❌ 音频编码失败: {e}")
        return None

def call_aog_stt(
    audio_base64: str,
    model: str = DEFAULT_MODEL,
    language: Optional[str] = None
) -> Optional[Dict]:
    """
    Call AOG Speech-to-Text API
    
    Args:
        audio_base64: base64 编码音频数据
        model: Model name
        language: 语言代码 (zh/en/等)
    
    Returns:
        API response data
    """
    try:
        url = f"{AOG_BASE_URL}/aog/{AOG_API_SPEC_VERSION}/services/speech-to-text"
        
        payload = {
            "model": model,
            "audio": audio_base64
        }
        
        if language:
            payload["language"] = language
        
        print(f"📤 Sending request...")
        if language:
            print(f"🌐 语言: {language}")
        
        response = requests.post(url, json=payload, timeout=120)
        response.raise_for_status()
        
        print("✅ Recognition successful")
        return response.json()
        
    except requests.exceptions.ConnectionError:
        print("❌ 无法连接到 AOG 服务")
        return None
    except requests.exceptions.Timeout:
        print("❌ Request timeout,音频文件可能过大")
        return None
    except Exception as e:
        print(f"❌ 错误: {e}")
        return None

def print_segments(response: Optional[Dict]):
    """
    格式化打印Recognition result
    
    Args:
        response: API response data
    """
    if not response:
        return
    
    segments = response.get("segments", [])
    
    if not segments:
        print("⚠️  没有Recognition result")
        return
    
    print("\n" + "="*60)
    print("📝 Recognition result:")
    print("="*60)
    
    for segment in segments:
        seg_id = segment.get("id", 0)
        start = segment.get("start", "")
        end = segment.get("end", "")
        text = segment.get("text", "")
        
        print(f"\n[{seg_id}] {start} --> {end}")
        print(f"    {text}")
    
    print("\n" + "="*60)
    
    # 打印完整文本
    full_text = " ".join([seg.get("text", "") for seg in segments])
    print("\n📄 完整文本:")
    print(full_text)
    print("\n" + "="*60 + "\n")

print("✅ Helper Functions定义完成")

## 3. Basic Speech Recognition

Note:需要准备一个测试音频文件

In [ ]:
# 基本示例
# 请将 'your_audio.wav' 替换为实际音频文件路径
audio_path = "your_audio.wav"

if Path(audio_path).exists():
    # 显示音频
    print("🔊 原始音频:")
    display(Audio(audio_path))
    
    # 转换为 base64
    audio_base64 = audio_to_base64(audio_path)
    
    if audio_base64:
        # 调用 API
        response = call_aog_stt(audio_base64)
        
        # 打印结果
        print_segments(response)
else:
    print(f"⚠️  音频文件不存在: {audio_path}")
    print("请准备一个测试音频文件并更新路径")

## 4. Multi-language Recognition

In [ ]:
# 中文识别
if Path(audio_path).exists():
    audio_base64 = audio_to_base64(audio_path)
    
    if audio_base64:
        print("🇨🇳 中文识别")
        response = call_aog_stt(audio_base64, language="zh")
        print_segments(response)

In [ ]:
# 英文识别
if Path(audio_path).exists():
    audio_base64 = audio_to_base64(audio_path)
    
    if audio_base64:
        print("🇺🇸 英文识别")
        response = call_aog_stt(audio_base64, language="en")
        print_segments(response)

## 5. Timeline Processing

提取和处理时间轴信息,可用于生成字幕等

In [ ]:
def generate_srt(segments: List[Dict], output_file: str = "subtitles.srt"):
    """
    生成 SRT 字幕文件
    
    Args:
        segments: Recognition result分段数组
        output_file: 输出文件名
    """
    try:
        with open(output_file, "w", encoding="utf-8") as f:
            for segment in segments:
                seg_id = segment.get("id", 0) + 1
                start = segment.get("start", "00:00:00.000").replace(".", ",")
                end = segment.get("end", "00:00:00.000").replace(".", ",")
                text = segment.get("text", "")
                
                f.write(f"{seg_id}\n")
                f.write(f"{start} --> {end}\n")
                f.write(f"{text}\n\n")
        
        print(f"✅ 字幕文件已生成: {output_file}")
    except Exception as e:
        print(f"❌ 生成字幕失败: {e}")

# 生成字幕示例
if Path(audio_path).exists():
    audio_base64 = audio_to_base64(audio_path)
    
    if audio_base64:
        response = call_aog_stt(audio_base64)
        
        if response and "segments" in response:
            generate_srt(response["segments"])
            
            # 显示字幕内容
            with open("subtitles.srt", "r", encoding="utf-8") as f:
                print("\n📄 生成字幕文件内容:")
                print("="*60)
                print(f.read())
                print("="*60)

## 🎉 Summary

Through this notebook, you have learned:

1. ✅ 调用 Speech-to-Text API
2. ✅ 处理音频文件(base64 编码)
3. ✅ Multi-language Recognition
4. ✅ 解析时间轴信息
5. ✅ 生成 SRT 字幕文件

### 使用技巧

- 🎤 使用清晰音频提高识别准确率
- 🌐 指定正确语言Args
- ⏱️ 利用时间轴信息生成字幕
- 📁 支持多种音频格式

### Next Steps

- 尝试不同语言音频
- 测试长音频文件
- Check out [实时语音识别示例](../speech-to-text-ws/)
- Check out [文本转语音示例](../text-to-speech/)
- Read [AOG API 文档](../../docs/)